In [2]:
from compressed_reconstruct import OCTReconstruction
import matplotlib.gridspec as gridspec
import cv2
from misc import processing
import matplotlib.pyplot as plt
import numpy as np
import math

MemoryError: Unable to allocate 512. GiB for an array with shape (262144, 262144) and data type float64

In [ ]:
# Init filename and address
file_name = ['ear']
short_axis_length = 20
# Module level constants
eps = 1e-14
lmbda = 1e-1

i = 0

In [ ]:
s, D = processing.load_data(file_name[i], decimation_factor=short_axis_length, data_only= False)
l2f, snorm = processing.to_l2_normed(s)

K = snorm.shape[1]  # number of A-line signal
N = snorm.shape[0]  # length of A-line signgal
M = 1  # state of dictionary
print("file name", file_name)

print("number of A-line signal", K)
print("length of A-line signgal", N)
rvmin, vmax = 5, 55  # dB
s_log = 20 * np.log10(abs(s))
s_log = processing.imag2uint(s_log, rvmin, vmax)

In [ ]:
fig = plt.figure(figsize=(18, 13), constrained_layout=True)
gs = gridspec.GridSpec(2, 3, figure=fig)

ax = fig.add_subplot(gs[1, 0])

vmin, vmax = 0,255
ax.set_title(file_name[i]+' %d dB-%d dB' % (vmax, vmin))
ax.imshow(s_log, cmap='gray', vmax=vmax, vmin=vmin)
ax.set_aspect(s_log.shape[1] / s_log.shape[0])
ax.set_axis_off()

ax = fig.add_subplot(gs[1, 1])
ax.set_title('estimated PSF')
ax.plot(abs(D))
ax.set_ylabel('magnitude(a.u.)')
ax.set_xlabel('axial depth(pixels)')

In [ ]:
cv2.imwrite('ear.jpg',s_log)

In [ ]:
speckle_weight = 0.1
lmbda = 0.05
w_lmbda = 0.05
x1, W = processing.make_sparse_representation(s, D, lmbda,w_lmbda, speckle_weight, Mask=True, Ear=True)
x1_log = 20 * np.log10(abs(x1))

In [ ]:
fig = plt.figure(figsize=(18, 13), constrained_layout=True)
gs = gridspec.GridSpec(2, 3, figure=fig)
ax = fig.add_subplot(gs[1, 0])


vmin, vmax = 0,255
ax.set_title(file_name[i]+' %d dB-%d dB' % (vmax, vmin))
ax.imshow(255*W, cmap='gray', vmax=vmax, vmin=vmin)
ax.set_aspect(x1_log.shape[1] / x1_log.shape[0])

plt.show()

In [ ]:
print(np.shape(s_log))
print(s_log)
print(np.amax(s_log))


In [ ]:
# Load the image file
# image_path = '../data/LK2_180.jpg'

# inputImg = plt.imread(image_path)
# inputImg = np.array(inputImg)
inputImg = s_log
print(np.shape(inputImg[:,1]))
# x is horizontal means col number

In [ ]:
axial_PSF = abs(D)
max_val = np.amax(axial_PSF)
focus_pos = np.argmax(axial_PSF)

# diameter = 14*0.59
diameter = 35*2
recon_size = 64
patch_size = 1
startX = 40
startY = 50
step_size = 2
step_interval = 14 # um
radius = (diameter/2)/(step_interval/step_size) 
z_depth = 0
print(radius)

In [ ]:
from compressed_reconstruct import OCTReconstruction


In [ ]:
recon = OCTReconstruction(recon_size=recon_size,radius=radius,step_size=step_size, z_0=focus_pos, estimated_axial_psf=axial_PSF)


In [ ]:
I_rec_norm, I_init_orig =recon.reconstruct_patch_psf_bscan(startX, startY, inputImg, W)

In [ ]:
# Plot the array as an image
plt.imshow(I_rec_norm, cmap='gray')
plt.colorbar()  # Add a colorbar for reference
plt.show()

In [ ]:
# Plot the array as an image
plt.imshow(I_init_orig, cmap='gray')
plt.colorbar()  # Add a colorbar for reference
plt.show()

In [ ]:
import cv2

In [ ]:
drawImg = cv2.cvtColor(np.array(inputImg), cv2.COLOR_GRAY2BGR)

# Draw a rectangle on the image
cv2.rectangle(drawImg, (startY, startX), (startY +17, startX + 17), (255, 0, 0), 1)
partial_img = drawImg[startX:startX+12, startY:startY+12]
# Display the image with the rectangle
plt.imshow(drawImg)
plt.show()

In [ ]:
# # Draw a rectangle on the image
# drawImg = np.array(inputImg)
# cv2.rectangle(drawImg, (startX, startY), (startX + 30, startY + 30), 0, 1)

# # Display the image with the rectangle
# # Plot the array as an image
# plt.imshow(drawImg, cmap='gray')
# plt.colorbar()  # Add a colorbar for reference
# plt.show()


In [ ]:
image_array = cv2.imread('output_image.jpg')
# Compute the minimum and maximum values of the array
min_value = np.min(image_array)
max_value = np.max(image_array)
print(max_value)

# Normalize the array and scale it to the range [0, 255]
normalized_array = (image_array - min_value) / (max_value - min_value) * 255

# Convert the normalized array to uint8 data type
uint8_array = normalized_array.astype('uint8')
# Display the image with the rectangle
plt.imshow(uint8_array)
plt.show()